<a href="https://colab.research.google.com/github/lacls/Vietnamese-SA-with-SOTA-approaches/blob/master/Trainer_Sents_UIT_VSFC_with%205%20models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install ftfy
!pip install transformers
!pip install vncorenlp
!pip install datasets
!pip install SentencePiece
!pip install pickle5


import pandas as pd
from torch.utils.data import *
from torch.nn import *
from transformers import *
import torch
import random,operator
import numpy as np
import os, re, string
import pickle, statistics
from ftfy import fix_text
from sklearn.metrics import *
import matplotlib.pyplot as plt
import math
import seaborn as sns
from datasets import load_metric
metric = load_metric('glue', "stsb") # Acccuracy
from transformers import logging
logging.set_verbosity_error()
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
!nvidia-smi

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from vncorenlp import VnCoreNLP
import os
from google.colab import drive
drive.mount('/content/drive')
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/Thesis/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 


def clean_doc(text, word_segment=False, lower_case=False):
  text= fix_text(text)
  if lower_case == True:
    text = text.lower()

  if word_segment == True:
    text = rdrsegmenter.tokenize(text)
    text = ' '.join([' '.join(x) for x in text])
    text = text.translate(text.maketrans('', '', string.punctuation.replace("_","")))
  else:
    text = text.translate(text.maketrans('', '', string.punctuation))
  text = re.sub('\\s+',' ',text)
  text = text.strip()
  return text

print(clean_doc("người ta có bạn bè nhìn vui thật %%%", word_segment=True, lower_case=True))
print(clean_doc("người ta có bạn bè nhìn vui thật ...", word_segment=False, lower_case=True))

class customDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

def show_confusion_matrix(confusion_matrix, model_name):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.savefig(path_save + 'confusion_matrix_'+model_name+'.jpg')
  plt.clf()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)
# Hyper parameter
epochs = 3
batch_size = 64
lr_rate = 5e-5


     |████████████████████████████████| 133kB 7.5MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219245 sha256=fd0ec0157c5a06c7a61e71bd4070f42a0ad36e72a2cb078f21065246fc56dea6
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
Wed Apr 14 15:29:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off

In [24]:
path_train = '/content/drive/MyDrive/Thesis/Data/uit_vsfc_train.csv'
path_valid = '/content/drive/MyDrive/Thesis/Data/uit_vsfc_dev.csv'
path_test = '/content/drive/MyDrive/Thesis/Data/uit_vsfc_test.csv'
path_save = '/content/drive/MyDrive/Thesis/Data/'
#------------------------------------------------------
import pickle5 as pickle
open_file=open("/content/drive/MyDrive/Thesis/Data_Character_Augment/uit_vsfc_train.pickle","rb")

data=pickle.load(open_file)
train_data=pd.read_csv(path_train)
#train_data = train_data[train_data[train_data['sentiment'].notna()]['text'].notna()]
append_Training_TRANSLATED=pd.DataFrame({'text':data,'label':list(train_data['label'])})
print(len(train_data))

train_data=train_data.append(append_Training_TRANSLATED[:round(len(append_Training_TRANSLATED)*0.4)])
train_data.reset_index(inplace=True,drop=True)
category_list=train_data['label'].unique().tolist()
num_labels = len(category_list)
print(category_list,num_labels)

valid_data=pd.read_csv(path_valid)
test_data=pd.read_csv(path_test)
print(len(train_data))


11426
[2, 0, 1] 3
15996


In [19]:
X_train = train_data["text"].apply(clean_doc,args=(True,False,)).tolist()
Y_train = train_data["label"].astype('int64').tolist()
print(len(X_train),len(Y_train))

X_valid = valid_data["text"].apply(clean_doc,args=(True,False,)).tolist()
Y_valid = valid_data["label"].astype('int64').tolist()
print(len(X_valid),len(Y_valid))

X_test = test_data["text"].apply(clean_doc,args=(True,False,)).tolist()
Y_test = test_data["label"].astype('int64').tolist()
print(len(X_test),len(Y_test))



22852 22852
1583 1583
3166 3166


NameError: ignored

In [ ]:
model_lists = "PhoBERT,XLMR,mBERT,viELECTRA_FPT,viBERT_FPT".split(",")
for name in model_lists:
    train_data = pd.read_csv(path_train)
    valid_data = pd.read_csv(path_valid)
    test_data = pd.read_csv(path_test)    
    
    print("=============================================================")
    print("Train")
    print(train_data["label"].value_counts())
    print("Validation")
    print(valid_data["label"].value_counts())
    print("Test")
    print(test_data["label"].value_counts())
    print("=============================================================")

    if name == "PhoBERT":
        
        model_name = "vinai/phobert-base"
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(category_list)num_labels=3)
        
    elif name == "viBERT_FPT":
        print("Loading viBERT_FPT ...")

        model_name = 'FPTAI/vibert-base-cased'
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(category_list))
        
    elif name == "viELECTRA_FPT":
        print("Loading viELECTRA_FPT ....")

     

        model_name = 'FPTAI/velectra-base-discriminator-cased'
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(category_list))
    
    elif name == "mBERT":
        print("Loading mBERT ...")
     
        model_name = 'bert-base-multilingual-cased'
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(category_list))
    
    elif name == "XLMR":
        print("Loading XLMR....")
        model_name = 'xlm-roberta-base'
        tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
        model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=(category_list))
    else:
          print("Error")

        
    train_encodings = tokenizer(X_train, truncation=True, padding=True)
    valid_encodings = tokenizer(X_valid, truncation=True, padding=True)
    test_encodings = tokenizer(X_test, truncation=True, padding=True)

    train_dataset = customDataset(train_encodings, Y_train)
    valid_dataset = customDataset(valid_encodings, Y_valid)
    test_dataset = customDataset(test_encodings, Y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    ###############################################################################
    args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy = "epoch",
        learning_rate=lr_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        warmup_steps=500,
        seed=seed,
    )

    trainer = Trainer(
        model=model,                         
        args=args,                           
        train_dataset=train_dataset,         
        tokenizer=tokenizer,
        eval_dataset=valid_dataset,
        compute_metrics=compute_metrics      
    )
    trainer.train()

    saved_trainer = copy.deepcopy(train)
    with open(f"{model_name}_{}.pkl", "wb") as output_file:
      pickle.dump(saved_trainer, output_file)

    preds = trainer.predict(test_dataset)[0]
    y_pred = np.argmax(preds, axis=1).flatten().tolist()

    micro_test = f1_score(Y_test, y_pred, average='micro')
    macro_test = f1_score(Y_test, y_pred, average='macro')
    weighted_test = f1_score(Y_test, y_pred, average='weighted')
    balance_accuracy_test = balanced_accuracy_score(Y_test,y_pred)
    accuracy_test = accuracy_score(Y_test,y_pred)

    reprort = classification_report(Y_test, y_pred)
    cm = confusion_matrix(Y_test, y_pred)
    
    
    df_cm = pd.DataFrame(cm, index=category_list, columns=category_list)
    show_confusion_matrix(df_cm,name)


    print()
    print("="*100)
    print("Results of model " + name)
    print("Accuracy: \t\t", round(accuracy_test*100,2))
    print("Balance Accuracy: \t", round(balance_accuracy_test*100,2))
    print("F1 weighted: \t\t", round(weighted_test*100,2))
    print("F1 macro: \t\t", round(macro_test*100,2))
    print("F1 micro: \t\t", round(micro_test*100,2))
    print(y_pred)
    print(Y_test)
    print(reprort)
    print("="*100)
    print()
    ###############################################################################
